In [1]:
import pandas as pd
import pickle
import re
import os
from os.path import join
from tqdm import tqdm
from datetime import datetime
wd = r'D:\0.전산\2.이관\써머리\data_230117'

In [11]:
df17 = pd.read_pickle(join(wd, "채무자.pkl"))
df18 = pd.read_pickle(join(wd, "채무자조회새창_20230119_1528.pkl"))

In [18]:
df17[~df17.채무상태.str.contains("신용회복")&~df17.채무상태.str.contains("종결") & df17.담당자.str.contains("신용회복")][["채무자키", "채권구분","성명","채무상태","담당자","보증인수"]]

,채무자키,채권구분,성명,채무상태,담당자,보증인수
28846,20408894,IRL,임은주,개인회생(면책),신용회복,1
35714,20411293,NPL,이봉희,파산(면책),신용회복,1
36175,20408147,NPL,임영일,파산(면책),신용회복,1
41114,20479932,CCRS-프리,최은정,개인회생(폐지),신용회복,0
43928,20460987,CCRS,변성남,사망,신용회복,1
44321,20460595,CCRS,김희정,파산(면책),신용회복,1
45756,20463197,CCRS,한병우,파산(면책),신용회복,1
45781,20460057,CCRS,김수복,파산(면책),신용회복,1
45870,20461028,CCRS,김효준,파산(면책),신용회복,1
47687,20462469,CCRS,강명주,파산(확정),신용회복,1
